# Developing Word Embeddings

Rather than use pre-trained embeddings (as we did in the sentence similarity baseline_deep_dive [notebook](../sentence_similarity/baseline_deep_dive.ipynb)), we can train word embeddings using our own dataset. In this notebook, we demonstrate the training process for producing word embeddings using the word2vec, GloVe, and fastText models. We'll utilize the STS Benchmark dataset for this task. 

# Table of Contents
* [Data Loading and Preprocessing](#Load-and-Preprocess-Data)
* [Word2Vec](#Word2Vec)
* [fastText](#fastText)
* [GloVe](#GloVe)
* [Concluding Remarks](#Concluding-Remarks)

In [19]:
!pip install gensim
!pip install googledrivedownloader
!python3 -m spacy download en_core_web_sm
!python -m spacy download en

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/en_core_web_sm
-->
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load

In [20]:
import gensim
import sys
import os

# Set the environment path
sys.path.append("../..")

import numpy as np
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
import en_core_web_sm
nlp = en_core_web_sm.load()
from utils_nlp.dataset import stsbenchmark
from utils_nlp.common.timer import Timer
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

In [21]:
# Set the path for where your repo is located
NLP_REPO_PATH = os.path.join('..','..')

# Set the path for where your datasets are located
BASE_DATA_PATH = os.path.join(NLP_REPO_PATH, "data")

# Set the path for location to save embeddings
SAVE_FILES_PATH = os.path.join(BASE_DATA_PATH, "trained_word_embeddings")
if not os.path.exists(SAVE_FILES_PATH):
    os.makedirs(SAVE_FILES_PATH)

## Load and Preprocess Data

In [13]:
# Produce a pandas dataframe for the training set
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")

# Clean the sts dataset
sts_train = stsbenchmark.clean_sts(train_raw)

100%|██████████| 401/401 [00:01<00:00, 238KB/s] 

Data downloaded to ../../data/raw/stsbenchmark


In [14]:
sts_train.head(5)

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [15]:
# Check the size of our dataframe
sts_train.shape

(5749, 3)

#### Training set preprocessing

In [30]:
def rm_spacy_stopwords(
    df,
    sentence_cols=["sentence1", "sentence2"],
    stop_cols=["sentence1_tokens_rm_stopwords", "sentence2_tokens_rm_stopwords"],
    custom_stopwords=[],
):
    """
    This function tokenizes the sentence pairs using spaCy and remove
    stopwords, defaulting to the spaCy en_core_web_sm model

    Args:
        df (pd.DataFrame): Dataframe with columns sentence_cols to tokenize.
        sentence_cols (list, optional): Column names for the raw sentence
        pairs.
        stop_cols (list, optional): Column names for the tokenized sentences
            without stop words.
        custom_stopwords (list of str, optional): List of custom stopwords to
            register with the spaCy model.

    Returns:
        pd.DataFrame: Dataframe with new columns stop_cols, each containing a
            list of tokens for their respective sentences.
    """
    nlp = en_core_web_sm.load()
    if len(custom_stopwords) > 0:
        for csw in custom_stopwords:
            nlp.vocab[csw].is_stop = True
    text_df = df[sentence_cols]
    nlp_df = text_df.applymap(lambda x: nlp(x))
    stop_df = nlp_df.applymap(lambda doc: [token.text for token in doc if not token.is_stop])
    stop_df.columns = stop_cols
    return pd.concat([df, stop_df], axis=1)

In [31]:
# Convert all text to lowercase
df_low = to_lowercase(sts_train)  
# Tokenize text
sts_tokenize = to_spacy_tokens(df_low) 
# Tokenize with removal of stopwords
sts_train_stop = rm_spacy_stopwords(sts_tokenize) 

In [32]:
# Append together the two sentence columns to get a list of all tokenized sentences.
all_sentences =  sts_train_stop[["sentence1_tokens_rm_stopwords", "sentence2_tokens_rm_stopwords"]]
# Flatten two columns into one list and remove all sentences that are size 0 after tokenization and stop word removal.
sentences = [i for i in all_sentences.values.flatten().tolist() if len(i) > 0]

In [33]:
len(sentences)

11498

In [34]:
sentence_lengths = [len(i) for i in sentences]
print("Minimum sentence length is {} tokens".format(min(sentence_lengths)))
print("Maximum sentence length is {} tokens".format(max(sentence_lengths)))
print("Median sentence length is {} tokens".format(np.median(sentence_lengths)))

Minimum sentence length is 1 tokens
Maximum sentence length is 43 tokens
Median sentence length is 6.0 tokens


In [41]:
sentences[:10]

[['plane', 'taking', '.'],
 ['air', 'plane', 'taking', '.'],
 ['man', 'playing', 'large', 'flute', '.'],
 ['man', 'playing', 'flute', '.'],
 ['man', 'spreading', 'shreded', 'cheese', 'pizza', '.'],
 ['man', 'spreading', 'shredded', 'cheese', 'uncooked', 'pizza', '.'],
 ['men', 'playing', 'chess', '.'],
 ['men', 'playing', 'chess', '.'],
 ['man', 'playing', 'cello', '.'],
 ['man', 'seated', 'playing', 'cello', '.']]

## Word2Vec

Word2vec is a predictive model for learning word embeddings from text (see [original research paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)). Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

The gensim Word2Vec model has many different parameters (see [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm; 1 for skip-gram and 0 for CBOW (defaults to 0)

In [42]:
# Set up a Timer to see how long the model takes to train
t = Timer()

In [46]:
t.start()

# Train the Word2vec model
word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=10, sg=0)

t.stop()

In [47]:
print("Time elapsed: {}".format(t))

Time elapsed: 0.9831


Now that the model is trained we can:

1. Query for the word embeddings of a given word. 
2. Inspect the model vocabulary
3. Save the word embeddings

In [45]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", word2vec_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words.
print("\nFirst 30 vocabulary words:", list(word2vec_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format.
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=True)  # binary format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=False)  # ASCII format

Embedding for apple: [ 0.24245308 -0.22766319  0.03921927 -0.00569307  0.11457231 -0.1936935
  0.02717012 -0.02499952 -0.08161789 -0.06850132 -0.12185523  0.09056653
 -0.16452083 -0.05633897  0.13712841  0.17568235 -0.03181924  0.11394875
 -0.08278044  0.00519803  0.20588045 -0.14170235  0.1036065  -0.18324244
  0.15329956  0.07836024  0.01878471 -0.10772411  0.07382355 -0.27275255
 -0.01688346 -0.06271663 -0.01128439 -0.13694242  0.00925762  0.09867323
 -0.09384014  0.04981905  0.08084494  0.05437228 -0.02023421 -0.11140446
 -0.01054097  0.16789724 -0.03027184  0.02410144 -0.10566749  0.21216291
 -0.13435307 -0.03032943  0.00840679  0.11766987 -0.16479416  0.0108357
  0.15942638  0.21357156  0.1415852   0.01696487 -0.06481776 -0.02605086
 -0.12410068  0.23055707 -0.18627672 -0.08867729  0.08526563  0.11027125
  0.1209132   0.05080904  0.02636342  0.12432348 -0.03489598  0.01239383
  0.0515947   0.04784227  0.0122715   0.04592336 -0.01565376  0.1566813
  0.17707448 -0.03667117  0.03794

## fastText

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings (see [original research paper](https://arxiv.org/pdf/1607.04606.pdf)). fastText is significantly different than word2vec or GloVe in that these two algorithms treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail.

The gensim fastText model has many different parameters (see [here](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm- 1 for skip-gram and 0 for CBOW (defaults to 0)
- iter: number of epochs (defaults to 5)


In [48]:
# Set up a Timer to see how long the model takes to train
t = Timer()

In [49]:
t.start()

# Train the FastText model
fastText_model = FastText(size=100, window=5, min_count=5, sentences=sentences, iter=30)

t.stop()

In [50]:
print("Time elapsed: {}".format(t))

Time elapsed: 13.8839


We can utilize the same attributes as we saw above for word2vec due to them both originating from the gensim package

In [51]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", fastText_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words.
print("\nFirst 30 vocabulary words:", list(fastText_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format.
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=True)  # binary format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=False)  # ASCII format

Embedding for apple: [-0.16347437 -0.7088631  -0.03814459  0.6433484   0.40757865 -0.13082083
 -0.93840206 -0.37719026  0.3627747   0.06398258  0.39715233  0.1359164
 -0.2876     -0.53517324 -0.53162557  0.06264706 -0.28391424 -0.16937332
 -0.05114771 -0.6231778   0.59394616  0.09203725  0.29574725 -0.16855736
  0.03266582 -0.07413924  0.43552378  0.21756841 -0.13717045  0.6044816
  0.08262639  0.06420626 -0.2889471   0.1491795  -0.273529    0.12348502
  0.43662795  0.35906202  0.07150669 -0.03047936  0.54101485  0.3310147
  0.20998567  0.18642022  0.07052571  0.15417226 -0.86126775  0.21909595
 -0.33046338  0.905908   -0.68186295 -0.09857658 -0.25510272  0.41998228
  0.4686534  -0.38099957 -0.14052735  0.07257152  0.07026441  0.8441788
 -0.06001268 -0.14090243 -0.32976252  0.4902379  -0.34447205  0.06513128
  0.49258104  0.29840803  0.4146131  -0.0017421  -0.0774308  -0.14005211
  0.48880437 -0.5291176  -0.40951633  0.21010408 -0.25720742 -0.07856242
  0.39267027 -0.14111987 -0.297353

## GloVe

GloVe is an unsupervised algorithm for obtaining word embeddings created by the Stanford NLP group (see [original research paper](https://nlp.stanford.edu/pubs/glove.pdf)). Training occurs on word-word co-occurrence statistics with the objective of learning word embeddings such that the dot product of two words' embeddings is equal to the words' probability of co-occurrence. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. 

Gensim doesn't have an implementation of the GloVe model and the other python packages that implement GloVe are unstable, so we leveraged the code directly from the Stanford NLP [repo](https://github.com/stanfordnlp/GloVe). 

In [52]:
# Define path
glove_model_path = os.path.join(NLP_REPO_PATH, "utils_nlp", "models", "glove")
# Execute shell commands
!cd $glove_model_path && make

mkdir -p build
gcc src/glove.c -o build/glove -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/glove.c:1:1: warning: C++ style comments are not allowed in ISO C90 [enabled by default]
 //  GloVe: Global Vectors for Word Representation
 ^
src/glove.c:1:1: warning: (this will be reported only once per input file) [enabled by default]
src/glove.c:57:6: warning: ISO C90 does not support ‘long long’ [-Wlong-long]
 long long num_lines, *lines_per_thread, vocab_size;
      ^
src/glove.c: In function ‘initialize_parameters’:
src/glove.c:67:10: warning: ISO C90 does not support ‘long long’ [-Wlong-long]
     long long a, b;
          ^
src/glove.c: In function ‘glove_thread’:
src/glove.c:105:10: warning: ISO C90 does not support ‘long long’ [-Wlong-long]
     long long a, b ,l1, l2;
          ^
src/glove.c:106:10: warning: ISO C90 does not support ‘long long’ [-Wlong-long]
     long long id = *(long long*)vid;
          ^
src/glove.c:106:27: warning: ISO C90 does no

### Train GloVe vectors

Training GloVe embeddings requires some data prep and then 4 steps (also documented in the original Stanford NLP repo [here](https://github.com/stanfordnlp/GloVe/tree/master/src)).

**Step 0: Prepare Data**
   
In order to train our GloVe vectors, we first need to save our corpus as a text file with all words separated by 1+ spaces or tabs. Each document/sentence is separated by a new line character.

In [53]:
# Save our corpus as tokens delimited by spaces with new line characters in between sentences.
training_corpus_file_path = os.path.join(SAVE_FILES_PATH, "training-corpus-cleaned.txt")
with open(training_corpus_file_path, 'w', encoding='utf8') as file:
    for sent in sentences:
        file.write(" ".join(sent) + "\n")

In [54]:
# Set up a Timer to see how long the model takes to train
t = Timer()
t.start()

**Step 1: Build Vocabulary**

Run the vocab_count executable. There are 3 optional parameters:
1. min-count: lower limit on how many times a word must appear in dataset. Otherwise the word is discarded from our vocabulary.
2. max-vocab: upper bound on the number of vocabulary words to keep
3. verbose: 0, 1, or 2 (default)

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the vocabulary to 

In [55]:
# Define path
vocab_count_exe_path = os.path.join(glove_model_path, "build", "vocab_count")
vocab_file_path = os.path.join(SAVE_FILES_PATH, "vocab.txt")
# Execute shell commands
!$vocab_count_exe_path -min-count 5 -verbose 2 <$training_corpus_file_path> $vocab_file_path

BUILDING VOCABULARY
Processed 0 tokens.Processed 85334 tokens.
Counted 11716 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 2943.



**Step 2: Construct Word Co-occurrence Statistics**

Run the cooccur executable. There are many optional parameters, but we list the top ones here:
1. symmetric: 0 for only looking at left context, 1 (default) for looking at both left and right context
2. window-size: number of context words to use (default 15)
3. verbose: 0, 1, or 2 (default)
4. vocab-file: path/name of the vocabulary file created in Step 1
5. memory: soft limit for memory consumption, default 4
6. max-product: limit the size of dense co-occurrence array by specifying the max product (integer) of the frequency counts of the two co-occurring words

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the co-occurrences to

In [56]:
# Define path
cooccur_exe_path = os.path.join(glove_model_path, "build", "cooccur")
cooccurrence_file_path = os.path.join(SAVE_FILES_PATH, "cooccurrence.bin")
# Execute shell commands
!$cooccur_exe_path -memory 4 -vocab-file $vocab_file_path -verbose 2 -window-size 15 <$training_corpus_file_path> $cooccurrence_file_path

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "../../data/trained_word_embeddings/vocab.txt"...loaded 2943 words.
Building lookup table...table contains 8661250 elements.
Processing token: 0Processed 85334 tokens.
Writing cooccurrences to disk......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.Merging cooccurrence files: processed 188154 lines.



**Step 3: Shuffle the Co-occurrences**

Run the shuffle executable. The parameters are as follows:
1. verbose: 0, 1, or 2 (default)
2. memory: soft limit for memory consumption, default 4
3. array-size: limit to the length of the buffer which stores chunks of data to shuffle before writing to disk

Then provide the path to the co-occurrence file we created in Step 2 followed by a file path that we'll save the shuffled co-occurrences to

In [57]:
# Define path
shuffle_exe_path = os.path.join(glove_model_path, "build", "shuffle")
cooccurrence_shuf_file_path = os.path.join(SAVE_FILES_PATH, "cooccurrence.shuf.bin")
# Execute shell commands
!$shuffle_exe_path -memory 4 -verbose 2 <$cooccurrence_file_path> $cooccurrence_shuf_file_path

SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 188154 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.188154 lines.Merging temp files: processed 188154 lines.



**Step 4: Train GloVe model**

Run the glove executable. There are many parameter options, but the top ones are listed below:
1. verbose: 0, 1, or 2 (default)
2. vector-size: dimension of word embeddings (50 is default)
3. threads: number threads, default 8
4. iter: number of iterations, default 25
5. eta: learning rate, default 0.05
6. binary: whether to save binary format (0: text = default, 1: binary, 2: both)
7. x-max: cutoff for weighting function, default is 100
8. vocab-file: file containing vocabulary as produced in Step 1
9. save-file: filename to save vectors to 
10. input-file: filename with co-occurrences as returned from Step 3

In [58]:
# Define path
glove_exe_path = os.path.join(glove_model_path, "build", "glove")
glove_vector_file_path = os.path.join(SAVE_FILES_PATH, "GloVe_vectors")
# Execute shell commands
!$glove_exe_path -save-file $glove_vector_file_path -threads 8 -input-file \
$cooccurrence_shuf_file_path -x-max 10 -iter 15 -vector-size 50 -binary 2 \
-vocab-file $vocab_file_path -verbose 2

TRAINING MODEL
Read 188154 lines.
Initializing parameters...done.
vector size: 50
vocab size: 2943
x_max: 10.000000
alpha: 0.750000
08/26/20 - 02:40.35AM, iter: 001, cost: 0.078540
08/26/20 - 02:40.35AM, iter: 002, cost: 0.072284
08/26/20 - 02:40.35AM, iter: 003, cost: 0.070181
08/26/20 - 02:40.35AM, iter: 004, cost: 0.066770
08/26/20 - 02:40.35AM, iter: 005, cost: 0.063463
08/26/20 - 02:40.35AM, iter: 006, cost: 0.060689
08/26/20 - 02:40.35AM, iter: 007, cost: 0.058074
08/26/20 - 02:40.35AM, iter: 008, cost: 0.056036
08/26/20 - 02:40.35AM, iter: 009, cost: 0.053891
08/26/20 - 02:40.35AM, iter: 010, cost: 0.051754
08/26/20 - 02:40.35AM, iter: 011, cost: 0.049567
08/26/20 - 02:40.36AM, iter: 012, cost: 0.047368
08/26/20 - 02:40.36AM, iter: 013, cost: 0.045185
08/26/20 - 02:40.36AM, iter: 014, cost: 0.043070
08/26/20 - 02:40.36AM, iter: 015, cost: 0.041049


In [59]:
t.stop()

In [60]:
print("Time elapsed: {}".format(t))

Time elapsed: 17.9333


### Inspect Word Vectors

Like we did above for the word2vec and fastText models, let's now inspect our word embeddings

In [61]:
#load in the saved word vectors.
glove_wv = {}
glove_vector_txt_file_path = os.path.join(SAVE_FILES_PATH, "GloVe_vectors.txt")
with open(glove_vector_txt_file_path, encoding='utf-8') as f:
    for line in f:
        split_line = line.split(" ")
        glove_wv[split_line[0]] = [float(i) for i in split_line[1:]]

In [62]:
# 1. Let's see the word embedding for "apple" by passing in "apple" as the key.
print("Embedding for apple:", glove_wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words.
print("\nFirst 30 vocabulary words:", list(glove_wv.keys())[:20])

Embedding for apple: [-0.044859, 6.8e-05, -0.022426, 0.026261, -0.06952, 0.029158, 0.01426, -0.039875, -0.025899, 0.138781, -0.062181, -0.001295, 0.174383, 0.080319, -0.05355, -0.006372, 0.065814, 0.046111, 0.017242, 0.011185, -0.08876, -0.020932, 0.033258, -0.099094, -0.044156, 0.016317, -0.025087, -0.033505, 0.261548, 0.099473, -0.025311, 0.085649, -0.08621, -0.106542, 0.100169, 0.071185, -0.049384, -0.183906, -0.058833, -0.225029, -0.031055, 0.229341, -0.003073, -0.128326, 0.094158, 0.104789, 0.045005, 0.001743, -0.047274, 0.133491]

First 30 vocabulary words: ['.', ',', 'man', '-', '"', 'woman', "'", 'said', 'dog', 'playing', ':', 'white', 'black', '$', 'killed', 'percent', 'new', 'syria', 'people', 'china']


# Concluding Remarks

In this notebook we have shown how to train word2vec, GloVe, and fastText word embeddings on the STS Benchmark dataset. We also inspected how long each model took to train on our dataset: word2vec took 0.39 seconds, GloVe took 8.16 seconds, and fastText took 10.41 seconds.

FastText is typically regarded as the best baseline for word embeddings (see [blog](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a)) and is a good place to start when generating word embeddings. Now that we generated word embeddings on our dataset, we could also repeat the baseline_deep_dive notebook using these embeddings (versus the pre-trained ones from the internet). 